In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import copy
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
def Stratification(data, ps):
    data_copy = data.copy()[['A', 'Y']]
    stratum = pd.qcut(x=ps, q=[0, .2, .4, .6, .8, 1.], labels=False)
    data_copy['stratum'] = stratum

    ATE = 0
    N = data_copy.shape[0]
    for j in range(5):
        data_new = data_copy[data_copy.stratum == j]
        Nj = data_new.shape[0]
        
        data_use1 = data_new[data_new['A'] == 1]
        N1j = data_use1.shape[0]
        s1 = sum(data_use1.A * data_use1.Y) / N1j
        
        data_use2 = data_new[data_new['A'] == 0]
        N0j = data_use2.shape[0]
        s2 = sum((1 - data_use2.A) * data_use2.Y) / N0j
        
        ATE = ATE + (Nj/N) * (s1 - s2)
    return ATE

In [3]:
highDim_data = pd.read_csv('../data/highDim_dataset.csv')
lowDim_data = pd.read_csv('../data/lowDim_dataset.csv')

X_low = lowDim_data.iloc[:,2:]
A_low = lowDim_data['A']

X_high = highDim_data.iloc[:,2:]
A_high = highDim_data['A']

In [4]:
X_train_low,X_test_low,A_train_low,A_test_low = train_test_split(X_low,
                                                         A_low,
                                                         test_size=.2,
                                                         random_state=123)

X_train_high,X_test_high,A_train_high,A_test_high = train_test_split(X_high,
                                                         A_high,
                                                         test_size=.2,
                                                         random_state=123)

In [5]:
#params_low = {'learning_rate':[0.01,0.1,0.2,0.5], 'n_estimators':[50,100,150,200]}
#gscv_low = GridSearchCV(AdaBoostClassifier(),params_low,cv=5).fit(X_train_low,A_train_low)
#gscv_low.best_params_

In [6]:
# {'learning_rate': 0.01, 'n_estimators': 50}

In [7]:
ada_low = AdaBoostClassifier(learning_rate = 0.01, n_estimators = 50).fit(X_train_low,A_train_low)

print(ada_low.score(X_train_low,A_train_low))
print(ada_low.score(X_test_low,A_test_low))

ps_low = ada_low.predict_proba(X_low)[:,1]
ps_low = pd.DataFrame(ps_low).rank(method='first')
ps_low = np.array(ps_low[0])

0.7975
0.85


In [8]:
Stratification(lowDim_data, ps_low)

3.618898696170567

In [9]:
params_high = {'learning_rate':[0.01,0.1,0.2,0.5], 'n_estimators':[100,150,200]}
gscv_high = GridSearchCV(AdaBoostClassifier(),params_high,cv=5).fit(X_train_high,A_train_high)
gscv_high.best_params_

{'learning_rate': 0.01, 'n_estimators': 150}

In [10]:
#{'learning_rate': 0.01, 'n_estimators': 150}

In [11]:
ada_high = AdaBoostClassifier(learning_rate = 0.01, n_estimators = 150).fit(X_train_high,A_train_high)

print(ada_high.score(X_train_high,A_train_high))
print(ada_high.score(X_test_high,A_test_high))

ps_high = ada_high.predict_proba(X_high)[:,1]
ps_high = pd.DataFrame(ps_high).rank(method='first')
ps_high = np.array(ps_high[0])

0.735625
0.695


In [12]:
Stratification(highDim_data, ps_high)

-60.836963965904644